In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

df = pd.read_csv('https://data.nasa.gov/api/views/gh4g-9sfh/rows.csv?accessType=DOWNLOAD')
df.to_csv('../Data/raw/Nasa_MeteoriteLanding.csv')
df

name     id nametype              recclass  mass (g)   fall  \
0          Aachen      1    Valid                    L5      21.0   Fell   
1          Aarhus      2    Valid                    H6     720.0   Fell   
2            Abee      6    Valid                   EH4  107000.0   Fell   
3        Acapulco     10    Valid           Acapulcoite    1914.0   Fell   
4         Achiras    370    Valid                    L6     780.0   Fell   
...           ...    ...      ...                   ...       ...    ...   
45711  Zillah 002  31356    Valid               Eucrite     172.0  Found   
45712      Zinder  30409    Valid  Pallasite, ungrouped      46.0  Found   
45713        Zlin  30410    Valid                    H4       3.3  Found   
45714   Zubkovsky  31357    Valid                    L6    2167.0  Found   
45715  Zulu Queen  30414    Valid                  L3.7     200.0  Found   

         year    reclat    reclong             GeoLocation  
0      1880.0  50.77500    6.08333       (50.775, 6.08333)  
1      1951.0  56.18333   10.23333    (56.18333, 10.23333)  
2      1952.0  54.21667 -113.00000      (54.21667, -113.0)  
3      1976.0  16.88333  -99.90000       (16.88333, -99.9)  
4      1902.0 -33.16667  -64.95000     (-33.16667, -64.95)  
...       ...       ...        ...                     ...  
45711  1990.0  29.03700   17.01850       (29.037, 17.0185)  
45712  1999.0  13.78333    8.96667     (13.78333, 8.96667)  
45713  1939.0  49.25000   17.66667       (49.25, 17.66667)  
45714  2003.0  49.78917   41.50460     (49.78917, 41.5046)  
45715  1976.0  33.98333 -115.68333  (33.98333, -115.68333)  

[45716 rows x 10 columns]

In [2]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45716 entries, 0 to 45715
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         45716 non-null  object 
 1   id           45716 non-null  int64  
 2   nametype     45716 non-null  object 
 3   recclass     45716 non-null  object 
 4   mass (g)     45585 non-null  float64
 5   fall         45716 non-null  object 
 6   year         45425 non-null  float64
 7   reclat       38401 non-null  float64
 8   reclong      38401 non-null  float64
 9   GeoLocation  38401 non-null  object 
dtypes: float64(4), int64(1), object(5)
memory usage: 3.5+ MB


id      mass (g)          year        reclat       reclong
count  45716.000000  4.558500e+04  45425.000000  38401.000000  38401.000000
mean   26889.735104  1.327808e+04   1991.828817    -39.122580     61.074319
std    16860.683030  5.749889e+05     25.052766     46.378511     80.647298
min        1.000000  0.000000e+00    860.000000    -87.366670   -165.433330
25%    12688.750000  7.200000e+00   1987.000000    -76.714240      0.000000
50%    24261.500000  3.260000e+01   1998.000000    -71.500000     35.666670
75%    40656.750000  2.026000e+02   2003.000000      0.000000    157.166670
max    57458.000000  6.000000e+07   2101.000000     81.166670    354.473330

In [3]:
df.isna().sum()

name              0
id                0
nametype          0
recclass          0
mass (g)        131
fall              0
year            291
reclat         7315
reclong        7315
GeoLocation    7315
dtype: int64

In [4]:
#Nuevas Categorias 
#Hemisferios
df['hemisphere_ns'] = df['reclat'].apply(lambda x: 'North' if x >= 0 else 'South')


df['hemisphere_ew'] = df['reclong'].apply(lambda x: 'East' if x >= 0 else 'West')

print(df[['reclat', 'reclong', 'hemisphere_ns', 'hemisphere_ew']].head())

     reclat    reclong hemisphere_ns hemisphere_ew
0  50.77500    6.08333         North          East
1  56.18333   10.23333         North          East
2  54.21667 -113.00000         North          West
3  16.88333  -99.90000         North          West
4 -33.16667  -64.95000         South          West


In [5]:
df.dropna(inplace=True)
df.isna().sum()

name             0
id               0
nametype         0
recclass         0
mass (g)         0
fall             0
year             0
reclat           0
reclong          0
GeoLocation      0
hemisphere_ns    0
hemisphere_ew    0
dtype: int64

In [ ]:
import geopandas as gpd
import pandas as pd
import earthpy as et
import shapely

In [6]:
import fiona
datos_poblacion = gpd.read_file('/workspaces/Proyecto_Final_DataScience/Data/ne_110m_populated_places.shp')

NameError: name 'gpd' is not defined

In [8]:
##Densidad de poblacion

datos_poblacion = gpd.read_file('raw/ne_110m_populated_places.shp')

NameError: name 'gpd' is not defined

In [ ]:
gdf_meteoritos = gdf_meteoritos.to_crs(epsg=4326)
datos_poblacion = gpd.read_file('raw/ne_110m_populated_places.shp')

In [9]:
from shapely.geometry import Point
## Crear columna in_city para ver si los meteoritos caen en ciudades (donde se concentra mayor pobl)
df['geometry'] = df.apply(lambda row: Point(row['reclong'], row['reclat']), axis=1)

gdf_meteoritos = gpd.GeoDataFrame(df, geometry='geometry')

gdf_meteoritos.crs = 'EPSG:4326'  
datos_poblacion = datos_poblacion.to_crs(gdf_meteoritos.crs)

meteoritos_en_ciudades = gpd.sjoin(gdf_meteoritos, datos_poblacion, how="left", predicate="intersects")

gdf_meteoritos['in_city'] = ~meteoritos_en_ciudades['name'].isnull()

print(gdf_meteoritos[['in_city']].head())


NameError: name 'gpd' is not defined

In [ ]:
from folium.plugins import MarkerCluster
from shapely.wkt import loads as wkt_loads 
df_clean = df.dropna(subset=['GeoLocation']).copy()

False